# Cours "Géomatique" - Analyse spatiale 
### Louis Maritaud
### louis.maritaud@unilim.fr
## Analyse des stations Vélib' par arrondissement à Paris

**Objectif** : Maîtriser les opérations spatiales de base avec GeoPandas dans un vrai projet

### Compétences visées
- Charger et manipuler des données géospatiales
- Réaliser des jointures spatiales
- Créer des zones tampons (buffers)
- Calculer des densités spatiales
- Produire des visualisations cartographiques

# Projet global : Analyser la répartition des stations Vélib' à Paris et repérer les densités par arrondissement

---
# Charger les données

In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from shapely.geometry import Polygon

# Configuration pour de meilleurs graphiques
%matplotlib inline
plt.style.use('seaborn-v0_8-darkgrid')

### Création des données de stations Vélib'

**Note** : Ces données ont été téléchargées depuis Open Data Paris.  


In [ ]:
data ="DATA/velib-emplacement-des-stations.geojson"

stations = gpd.read_file(
    data)

print(f"Nombre de stations : {len(stations)}")
print("\nAperçu des données :")
stations.head()

### Création des arrondissements de Paris

**Note** : En pratique, charger depuis OpenStreetMap ou data.gouv.fr.     
Je vous ai déja téléchargé le bon fichier depuis Open Data Paris

In [ ]:
# Arrondissements de Paris
arrondissements = gpd.read_file("DATA/arrondissements.geojson")

print("Arrondissements créés :")
arrondissements

---
## Étape 2 : Visualiser les données

In [ ]:
# Carte des arrondissements et stations
fig, ax = plt.subplots(figsize=(12, 10))

arrondissements.plot(ax=ax, color='lightgray', edgecolor='black', alpha=0.5)
stations.plot(ax=ax, color='red', markersize=5, alpha=0.6)

plt.title("Stations Vélib' à Paris", fontsize=16, fontweight='bold')
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.tight_layout()
plt.show()

---
## Étape 3 : Compter les stations par arrondissement

### Jointure spatiale

La jointure spatiale permet d'associer chaque station à son arrondissement en fonction de sa position géographique.

In [ ]:
# Jointure spatiale : associer chaque station à son arrondissement
stations_avec_arrond = gpd.sjoin(stations, arrondissements, 
                                  how="left", predicate="within")

print("Aperçu des stations avec leur arrondissement :")
stations_avec_arrond.dropna(subset=["geom_x_y"], inplace=True)
stations_avec_arrond

### Calcul du nombre de stations par arrondissement

In [ ]:
# Compter le nombre de stations par arrondissement
nb_stations = stations_avec_arrond.groupby('l_ar').size()
print("\nNombre de stations par arrondissement :")
print(nb_stations)

# Ajouter cette info aux arrondissements
arrondissements = arrondissements.merge(
    nb_stations.rename('nb_stations'),
    left_on='l_ar',
    right_index=True,
    how='left'
)

print("\nArrondissements avec nombre de stations :")
arrondissements

### Visualisation choroplèthe

In [ ]:
# Visualiser
fig, ax = plt.subplots(figsize=(12, 10))
arrondissements.plot(column='nb_stations', 
                     cmap='Reds', 
                     legend=True,
                     edgecolor='black',
                     ax=ax)
stations.plot(ax=ax, color='blue', markersize=3, alpha=0.4)
plt.title("Nombre de stations Vélib' par arrondissement", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Étape 4 : Buffer de 500m autour des stations

### Reprojection en Lambert 93

Pour travailler en mètres, nous devons reprojeter nos données du système WGS84 (EPSG:4326) vers Lambert 93 (EPSG:2154).

In [ ]:
# Projeter en Lambert 93 pour travailler en mètres
stations_proj = stations.to_crs("EPSG:2154")
arrondissements_proj = arrondissements.to_crs("EPSG:2154")

print("Projection effectuée :")
print(f"CRS original : {stations.crs}")
print(f"Nouveau CRS : {stations_proj.crs}")

### Création des zones tampons (buffers)

In [ ]:
# Créer un buffer de 500m autour de chaque station
stations_buffer = stations_proj.copy()
stations_buffer['geometry'] = stations_proj.buffer(500)  # 500 mètres

print(f"Buffers créés : {len(stations_buffer)} zones de couverture")

### Visualisation des zones de couverture

In [ ]:
# Visualiser
fig, ax = plt.subplots(figsize=(12, 10))
arrondissements_proj.plot(ax=ax, color='lightgray', edgecolor='black')
stations_buffer.plot(ax=ax, color='blue', alpha=0.2, edgecolor='blue')
stations_proj.plot(ax=ax, color='red', markersize=5)

plt.title("Zones de couverture Vélib' (rayon 500m)", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

### Zone totale couverte

Fusion de tous les buffers pour visualiser la couverture globale.

In [ ]:
# Fusionner tous les buffers pour voir la zone totale couverte
from shapely.ops import unary_union
zone_couverte = unary_union(stations_buffer.geometry)

fig, ax = plt.subplots(figsize=(12, 10))
arrondissements_proj.plot(ax=ax, color='lightgray', edgecolor='black')
gpd.GeoSeries([zone_couverte]).plot(ax=ax, color='blue', alpha=0.3)
plt.title("Zone totale couverte par Vélib'", fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

---
## Densité de stations

### Calcul de la superficie des arrondissements

In [ ]:
# Calculer la superficie de chaque arrondissement (en km²)
arrondissements_proj['superficie_m2'] = arrondissements_proj.geometry.area
arrondissements_proj['superficie_km2'] = arrondissements_proj['superficie_m2'] / 1_000_000

print("Superficies calculées :")
print(arrondissements_proj[['l_ar', 'superficie_km2']])

### Calcul de la densité (stations par km²)

In [ ]:
# Calculer la densité (stations par km²)
arrondissements_proj['densite'] = (
    arrondissements_proj['nb_stations'] / arrondissements_proj['superficie_km2']
)

print("Tableau récapitulatif :")
print(arrondissements_proj[['l_ar', 'nb_stations', 
                             'superficie_km2', 'densite']])

### Carte de la densité

In [ ]:
# Carte de la densité
fig, ax = plt.subplots(figsize=(12, 10))
arrondissements_proj.plot(column='densite', 
                          cmap='YlOrRd', 
                          legend=True,
                          edgecolor='black',
                          legend_kwds={'label': "Densité (stations/km²)"},
                          ax=ax)

plt.title("Densité de stations Vélib' par arrondissement", fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

### Statistiques descriptives

In [ ]:
# Statistiques
print(f"\nDensité moyenne : {arrondissements_proj['densite'].mean():.2f} stations/km²")
print(f"Densité max : {arrondissements_proj['densite'].max():.2f} stations/km²")
print(f"Arrondissement le plus dense : {arrondissements_proj.loc[arrondissements_proj['densite'].idxmax(), 'l_ar']}")

## Centroïdes et noms d'arrondissements

In [ ]:
# On calcule les centroîdes des arrondissements 
arrondissements_proj["centroid"]=arrondissements_proj.geometry.centroid


fig, ax = plt.subplots(figsize=(12, 10))
arrondissements_proj.plot(column='densite', 
                          cmap='YlOrRd', 
                          legend=True,
                          edgecolor='black',
                          legend_kwds={'label': "Densité (stations/km²)"},
                          ax=ax)


# On ajoute les noms d'arrondissements à notre plot précédent
# Ajouter les labels au centre de chaque département
for idx, row in arrondissements_proj.iterrows():
    plt.annotate(
        text=row['l_ar'], 
        xy=(row['centroid'].x, row['centroid'].y),
        horizontalalignment='center',
        fontsize=9,
        color='black'
    )

plt.title("Densité de stations Vélib' par arrondissement", fontsize=16, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

---
## Exercices bonus

C'est à vous :

### 1. Capacité totale par arrondissement
Calculer la somme des capacités de toutes les stations de chaque arrondissement.

In [ ]:
# Votre code ici


### 2. Arrondissements mal desservis
Identifier les arrondissements avec une densité inférieure à la moyenne.

In [ ]:
# Votre code ici


### 3. Zones non couvertes
Identifier les zones situées à plus de 500m d'une station.

In [ ]:
# Votre code ici


---
## Conclusion

Dans ce cours, vous avez appris à :
- Manipuler des données géospatiales avec GeoPandas
- Effectuer des jointures spatiales
- Créer des zones tampons (buffers)
- Calculer des densités spatiales
- Produire des visualisations cartographiques

### Pour aller plus loin
- Analyser l'évolution temporelle de l'utilisation des stations
- Comparer avec d'autres modes de transport (métro, bus)
- En faire une carte interactive avec Folium